# 2022 Hunga Tonga Eruption
Written by Sage Lichtenwalner, Rutgers University, June 3, 2024

**Note this activity is still being developed**

On January 15, 2022, the [Hunga Tonga–Hunga Haʻapai volcano](https://en.wikipedia.org/wiki/2022_Hunga_Tonga–Hunga_Haʻapai_eruption_and_tsunami) erupted in the southern Pacific Ocean.  The ensuing erruption created an undertwater tsunami that crossed the entire Pacific Ocean.  The explosion was so severe, the eruption also caused the largest ever recored atmospheric shockwave that propagated around the globe.  

A number of ocean tidal and underwater pressure gauges around the Pacific ocean detected the tsunami.  And atmospheric pressure sensors around the world were also above to detect the pressure waves that resulted from the eruption.  The **Ocean Observatories Initiative (OOI)** is one of many observatories that operates several high-resolution atmospheric and ocean pressure sampling instruments.  The OOI includes the Endurance Array off the coast of Oregon in the Northeast Pacific, and the Pioneer NES array in the Mid-Atlantic.  

So the question is, did the OOI measure the impact of the volcanic eruption at it's sites thousands of miles away?

In this notebook, we will demonstrate how to access and plot data from the two coastal OOI arrays, in the Northweast Pacific and off the New England coast, to see if we can detect the eruption's signature at these locations.

In [ ]:
# Install erddapy
# !pip install erddapy

In [ ]:
# Notebook Setup
from erddapy import ERDDAP
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Setup Erddap connection
server = "https://erddap.dataexplorer.oceanobservatories.org/erddap/"
e = ERDDAP(
    server=server,
    protocol="tabledap",
    response="csv",
)

## Detecting a Tsunami with Seafloor Pressure
First, let's take a look at the seafloor pressure data at the Endurance Array.

In [ ]:
# Load CE04 Bottom Pressure
url_ce04_bot = e.get_download_url(
    dataset_id = "ooi-ce04osbp-lj01c-06-ctdbpo108",
    constraints = {
      "time>=": "2022-01-15",
      "time<=": "2022-01-17",
    }
)
# print(url_ce04_bot)
ce04_bot = pd.read_csv(url_ce04_bot, index_col='time', parse_dates=True, skiprows=[1])
ce04_bot.head()

In [ ]:
ce04_bot['sea_water_pressure'].plot()
plt.ylabel('Water Pressure (db)')  
plt.title('Oregon Offshore Cabled Benthic Experiment Package (CE04OSBP)')

Quick Quesions
* What do you think causes the general pattern you see in the pressure dataset?
* Can you see the impact of the tsunami in the pressure daset?  What does it look like?
* When did the tsunami first reach this station?

To answer this last question, we really need to zoom in on the data a bit more.  We could create a new plot with a narrower time range, or we can try using the [Plotly](https://plotly.com/python/time-series/) library to create an interactive graph.


In [ ]:
# !pip install plotly
import plotly.express as px

In [ ]:
px.line(x=ce04_bot.index, y=ce04_bot.sea_water_pressure, 
        labels=dict(y='Waer Pressure (db)'),
        title='Oregon Offshore Cabled Benthic Experiment Package (CE04OSBP)')

## Calculating Wave Speed
Now that you have the time the tsunami reached the Endurance Array, you can calculate the phase speed of the tsunami wave.

Phase Speed = Distance between Hunga and Endurance / Time it took to reach Endurance from Hunga

Becuse tsunamis act as *shallow water waves* in the ocean, you can calculate their expected phase speed using the shallow water wave equation.

C_(phase) = sqrt(g * H)

If we assume the height/depth of the ocean is the [average depth of the Pacific Ocean](https://www.britannica.com/place/Pacific-Ocean/Tides) (4,280), how does this estimate of the phase speed compare with what you calculated for the actual tsunami?

## Atmospheric Pressure Waves
One of the interesting impacts of the eruption was the massive atmospheric pressure wave it also caused.  Can we see this signal at the OOI arrays?

In [ ]:
# Load CE04 Surface Pressure
url_ce04_met = e.get_download_url(
    dataset_id = "ooi-ce04ossm-sbd11-06-metbka000",
    constraints = {
      "time>=": "2022-01-15",
      "time<=": "2022-01-17",
    }
)
# print(url_wave)

ce04_met = pd.read_csv(url_ce04_met, index_col='time', parse_dates=True, skiprows=[1])
ce04_met.head()

In [ ]:
ce04_met['air_pressure'].plot()
plt.ylabel('Air Pressure (mb)');
plt.title('Oregon Offshore Surface Mooring (CE04OSSM)');

## Calculating the Atm Wave Speed
In the air, pressure waves typically travel at the [speed of sound](https://en.wikipedia.org/wiki/Speed_of_sound), or about 343 m/s.

Calculate the distance between Hunga and this station, and the time it took for the wave to travel, and then compare this with the expected speed.

## Combining the plots
Let's plot both datasets on one graph to see how the timing of the atmospheric and seawater waves compares with each other.

Note, due to the way [pandas handles date ticks](https://pandas.pydata.org/docs/user_guide/visualization.html#suppressing-tick-resolution-adjustment), we will *resample* the datsets to have the same timestaps so we can plot them both on the same graph.

In [ ]:
fig,ax1 = plt.subplots()

color = 'tab:red'
ce04_met['air_pressure'].resample('1min').mean().plot(color=color, ax=ax1)
ax1.set_ylabel('Air Pressure (mb)', color=color)
ax1.tick_params(axis='y', labelcolor=color)

ax1b = ax1.twinx()  # instantiate a second axes that shares the same x-axis
color = 'tab:blue'
ce04_bot['sea_water_pressure'].resample('1min').mean().plot(color=color, ax=ax1b)
ax1b.set_ylabel('Water Pressure (db)', color=color)  
ax1b.tick_params(axis='y', labelcolor=color)

## What about the Pioneer Array?
Let's take a look at the atmospheric pressure at the Pioneer Array on the US East Coast to see if the impact of teh eruption was measured there.

In [ ]:
# Load CE04 Surface Pressure
url_cp01_met = e.get_download_url(
    dataset_id = "ooi-cp01cnsm-sbd11-06-metbka000",
    constraints = {
      "time>=": "2022-01-15",
      "time<=": "2022-01-17",
    }
)
# print(url_wave)

cp01_met = pd.read_csv(url_cp01_met, index_col='time', parse_dates=True, skiprows=[1])
cp01_met.head()

In [ ]:
cp01_met['air_pressure'].plot()
plt.ylabel('Air Pressure (mb)');
plt.title('Pioneer Array - Central Surface Mooring (CP01CNSM)');

Based on this data, calculte the speed of the wave that reached the Pioneer Array.  How does this compare with the expected speed, whcih shoudl be about the speed of sound.

*Note, these will be different, becuase (I think) the wave we observe at teh Pioneer Array is actually the reflection from the anti-pode.  If you recalculate the distance from Hunga to the Antipode, and then from there to Pioneer, how does that match?*


## Graphing all 3 datasets

In [ ]:
fig,(ax1,ax2,ax3) = plt.subplots(3,1, figsize=(10,6), sharex=True)

ce04_bot['sea_water_pressure'].resample('1min').mean().plot(ax=ax1)
ax1.set_ylabel('Water Pressure (db)')  
ax1.set_title('Oregon Offshore Cabled Benthic Experiment Package (CE04OSBP)')

ce04_met['air_pressure'].resample('1min').mean().plot(ax=ax2)
ax2.set_ylabel('Air Pressure (mb)')
ax2.set_title('Oregon Offshore Surface Mooring (CE04OSSM)')

cp01_met['air_pressure'].resample('1min').mean().plot(ax=ax3)
ax3.set_ylabel('Air Pressure (mb)')
ax3.set_title('Pioneer Array - Central Surface Mooring (CP01CNSM)');


## Extension Ideas
* The Endurance and Pioneer arrays include several atmospheric pressure sensors and underwater tide or pressure gagues.  Can you identify other senors that detected the tsunami or shockwave.
* Based on these addtional sensors, is your estimate of the speed the same?
* Can you predict when the tsunami would have hit Station Papa?  Can you identify the tsunami in the dataset?
* (advanced) Can you predict when the atmospheric shockwave might have hit Irminger Sea station? Can you identify the shockwave in the dataset?

## References
First a bunch from EOS...
* 6/26/17 - [New Volcanic Island Unveils Explosive Past](https://eos.org/science-updates/new-volcanic-island-unveils-explosive-past)
* 3/31/20 - [Satellite Sleuthing Detects Underwater Eruptions](https://eos.org/science-updates/satellite-sleuthing-detects-underwater-eruptions)
* ** 1/21/22 - [The Surprising Reach of Tonga’s Giant Atmospheric Waves](https://eos.org/articles/the-surprising-reach-of-tongas-giant-atmospheric-waves)
4/21/22 - [Tonga Eruption Made Waves in Earth’s Ionosphere](https://eos.org/research-spotlights/tonga-eruption-made-waves-in-earths-ionosphere)
* ** 5/12/22 - [Modeling Atmospheric Waves from Hunga Tonga–Hunga Ha‘apai](https://eos.org/research-spotlights/modeling-atmospheric-waves-from-hunga-tonga-hunga-haapai)
* 7/6/22 - [Dynamics of Volcanic Processes](https://eos.org/editors-vox/dynamics-of-volcanic-processes)
* 8/10/22 - [Tonga Volcanic Eruption Produced Ionospheric Hole and ‘Bubbles’](https://eos.org/editor-highlights/tonga-volcanic-eruption-produced-ionospheric-hole-and-bubbles)
* 10/24/22 - [Tracking Water in the Tongan Volcano’s Massive Eruption Plume](https://eos.org/research-spotlights/tracking-water-in-the-tongan-volcanos-massive-eruption-plume)
* 11/4/22 - [Looking to the Sky for Better Tsunami Warnings](https://eos.org/opinions/looking-to-the-sky-for-better-tsunami-warnings)
* 12/14/22 - [Social Media Posts Reveal Human Responses to Deadly Tongan Eruption](https://eos.org/articles/social-media-posts-reveal-human-responses-to-deadly-tongan-eruption)

** Definitely start with these.  And I highly recommend clicking through to the article on the 2nd one and downloading the movie in the supplemental info.

* 3/29/22 R-Bloggers - [Using R to detect the pressure wave from the 2022 Hunga Tonga eruption in personal weather station data](https://www.r-bloggers.com/2022/03/using-r-to-detect-the-pressure-wave-from-the-2022-hunga-tonga-eruption-in-personal-weather-station-data/) My dad sent this one to me - I'm more of a python guy ;)  

* 4/14/22 - NY Times - [It’s Super Spectacular.’ See How the Tonga Volcano Unleashed a Once-in-a-Century Shockwave](https://www.nytimes.com/interactive/2022/04/14/upshot/tonga-pressure-wave.html?unlocked_article_code=BXzP1PdwKMEyDz5JpHMk8piiief7vY8Inqq45cquJJzIXMY-_9PM1HeuXVDELGdLtaOOpSHYZnnA2ZIN6ymupdmlzqOmSYHlF3_9WpQ1xGLI8gwIKoA0_HKEwOuD23WEYoeoF5vZqHmjwK_SHTBLLfdP4Z9_cbM7by000oa8-RzNNfP6L8hUww1nzlISZh8gc_QM7Io2lP3bC46esNfGZAeDrnUlz96PEmSVIYH636cC0zaWh48_Yg82M_-Llfoh8NOdAbw-RgqOE7DGr0J_czePBK3IMrXg4lGoQ4yqm6R4toT6dZ46-ReQk8gU44Mnm6sxMeQUgiMOhbReaC3c8d6sPgf3FLaa3QJEng&smid=share-url) 

* 8/29/22 - Science News - [The Tonga eruption may have spawned a tsunami as tall as the Statue of Liberty](https://www.sciencenews.org/article/tonga-volcano-eruption-tsunami-shock-wave)

Finally, here are some posts on Flowing Data, which collects some of the best data visualizations from around the web.
* 1/18/22 - [Global ripple effect from underwater volcano](https://flowingdata.com/2022/01/18/global-ripple-effect-from-underwater-volcano/)
* 1/24/22 - [Scale of the Tonga eruption](https://flowingdata.com/2022/01/24/scale-of-the-tonga-eruption/)
* 4/15/22 - [Tonga shockwave around the world](https://flowingdata.com/2022/04/15/tonga-shockwave-around-the-world/)
* 11/25/22 - [Depth of the underwater Tonga volcano](https://flowingdata.com/2022/11/25/depth-of-the-underwater-tonga-volcano/)
